In [1]:
import datetime as dt
import numpy as np 
import pandas as pd
import argparse
from pathlib import Path
from pyspark import SparkContext
from pyspark.sql.session import SparkSession

In [2]:
input_file = './input/complaints_sample.csv'
output_path = 'output/'

In [3]:
sc = SparkContext()
spark = SparkSession(sc)

In [130]:
complaint_info = sc.textFile(input_file, use_unicode=True).cache()
print("Number of partitions: ", complaint_info.getNumPartitions())

Numbe of partitions:  2


In [131]:
def extract_complaints(partId, list_of_records):
    if partId==0: 
        next(list_of_records) # skipping the header line
    import csv
    reader = csv.reader(list_of_records)
    for row in reader:
        year_received = dt.datetime.strptime(row[0], '%m/%d/%Y').year
        product = row[1].lower()
        company = row[7].lower()            
        yield ((product, year_received, company), 1)

complaints = complaint_info.mapPartitionsWithIndex(extract_complaints)

In [161]:
def toCSVLine(entry):
    product = entry[0][0]
    if ',' in entry[0][0]:
        product = '"' + entry[0][0] + '"'
    return product + "," + str(entry[0][1]) + "," + str(entry[1][0]) + "," + str(entry[1][1]) + "," + str(entry[1][2])

In [162]:
# ( (product, year, company), 1)
# ( (product, year) (company, total_company_complaints) )
# 
# ...
# ( (product, year), (total_complaints, total_companies, comp.with.highest.complaints.% ) )

temp = complaint_info.mapPartitionsWithIndex(extract_complaints) \
    .reduceByKey(lambda x,y: x+y) \
    .map(lambda x: ( (x[0][0],x[0][1]), [x[1]] ) ) \
    .reduceByKey(lambda x,y: x+y) \
    .map(lambda x: ( x[0], ( sum(x[1]), len(x[1]), round(max(x[1])*100/sum(x[1])) ) ) ) \
    .sortByKey() \
    .map(toCSVLine) \
    .collect()


In [163]:
for t in temp:
    print(t)

bank account or service,2015,1,1,100
bank account or service,2016,2,2,50
checking or savings account,2017,1,1,100
checking or savings account,2018,20,10,25
checking or savings account,2019,461,72,13
checking or savings account,2020,3,3,33
consumer loan,2015,1,1,100
consumer loan,2016,1,1,100
consumer loan,2017,1,1,100
credit card,2016,4,4,25
credit card,2017,1,1,100
credit card or prepaid card,2017,1,1,100
credit card or prepaid card,2018,27,12,33
credit card or prepaid card,2019,437,42,15
credit card or prepaid card,2020,13,10,23
"credit reporting, credit repair services, or other personal consumer reports",2017,7,5,29
"credit reporting, credit repair services, or other personal consumer reports",2018,238,22,56
"credit reporting, credit repair services, or other personal consumer reports",2019,3114,203,50
"credit reporting, credit repair services, or other personal consumer reports",2020,144,10,51
debt collection,2015,4,3,50
debt collection,2016,10,4,60
debt collection,2017,13,11,15
d

In [150]:
temp.saveAsTextFile("output")

Py4JJavaError: An error occurred while calling o3549.saveAsTextFile.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.internal.io.SparkHadoopWriter$.write(SparkHadoopWriter.scala:100)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply$mcV$sp(PairRDDFunctions.scala:1096)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1094)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1094)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:385)
	at org.apache.spark.rdd.PairRDDFunctions.saveAsHadoopDataset(PairRDDFunctions.scala:1094)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopFile$4.apply$mcV$sp(PairRDDFunctions.scala:1067)
	at org.apache.spark.rdd.PairRDD